Scraping du site "https://www.techno-science.net/definition/12174.html" pour récupérer les aéroports de France

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "https://www.techno-science.net/definition/12174.html"
response = requests.get(url)
html_content = response.text

soup = BeautifulSoup(html_content, 'html.parser')
table = soup.find('table', {'class': 'wikitable'})

rows = table.find_all('tr')
data = []
for row in rows:
    cols = row.find_all('td')
    if cols:
        # Extraction des colonnes 'Code AITA', 'Nom de l'aérodrome' et 'Ville desservie'
        data.append([cols[0].text.strip(), cols[2].text.strip(), cols[3].text.strip()])

column_names = ["Code AITA", "Nom de l'aérodrome", "Ville desservie"]
df = pd.DataFrame(data, columns=column_names)

#print(df)


Code pour récupérer dans le dataframe le code AITA si on lui donne la ville

In [ ]:
import pandas as pd


def find_aita_code(city_name, df):
    result = df[df['Ville desservie'].str.contains(city_name, case=False, na=False)]
    if not result.empty:
        return result.iloc[0]['Code AITA']
    else:
        return None


Scraping de carbonfootprint pour récuperer l'empreinte carbonne en avion de deux villes

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
import re

def wait_for_element(by, value, timeout=10):
    wait = WebDriverWait(driver, timeout)
    return wait.until(EC.element_to_be_clickable((by, value)))

# Demander à l'utilisateur de saisir les villes de départ et d'arrivée
depart_city = input("Entrez la ville de départ : ")
arrive_city = input("Entrez la ville d'arrivée : ")

depart_aita_code = find_aita_code(depart_city, df)
arrive_aita_code = find_aita_code(arrive_city, df)

if depart_aita_code and arrive_aita_code:
    geckodriver_path = "C:/Users/noelt/Downloads/geckodriver-v0.34.0-win64/geckodriver.exe"
    service = Service(geckodriver_path)
    driver = webdriver.Firefox(service=service)
    driver.get("https://calculator.carbonfootprint.com/calculator.aspx?tab=3")

    arrival_field = wait_for_element(By.ID, "ctl05_rcbAirportTo_Input")
    arrival_field.clear()
    arrival_field.send_keys(arrive_aita_code)
    arrival_field.send_keys(Keys.TAB)

    arrow_element = wait_for_element(By.ID, "ctl05_rcbAirportTo_Arrow")
    arrow_element.click()

    departure_field = wait_for_element(By.ID, "ctl05_rcbAirportFrom_Input")
    departure_field.clear()
    departure_field.send_keys("PAR")
    departure_field.send_keys(Keys.TAB)

    arrow_element_departure = wait_for_element(By.ID, "ctl05_rcbAirportFrom_Arrow")
    arrow_element_departure.click()

    actions = ActionChains(driver)
    calculate_button = wait_for_element(By.ID, "ctl05_btnAddFlight")
    actions.move_to_element(calculate_button).click().perform()

    try:
        WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.CSS_SELECTOR, "table.footprints"))
        )
        carbon_footprint_element = driver.find_element(By.CSS_SELECTOR, "table.footprints td:first-child")
        carbon_footprint_text = carbon_footprint_element.text
        match = re.search(r"([\d.]+)\s*metric tons", carbon_footprint_text)
        carbon_footprint = match.group(1) if match else "Non trouvé"
    except Exception as e:
        print("Une erreur est survenue :", e)

    print("L'empreinte carbone est :", carbon_footprint, "tonnes métriques de CO2e")

    #driver.quit()
else:
    print("Les codes AITA pour les villes fournies n'ont pas été trouvés.")
